In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [2]:
# reading html files from folder

In [3]:
data = []

#'Riv_Shelter_HTML_Files'is the name for the shelter HTML Files
#'Riv_Food_Pantry_HTML_Files'is the name for the food pantry HTML Files
#'Riv_Mental_Health_HTML_Files'is the name for the shelter HTML Files
folder_path = 'Riv_Mental_Health_HTML_Files'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            
            if availability_element:
                availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                availability = "Not specified"

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            
            if cost_element:  # Check if cost_element exists
                cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                cost = "Not specified"  # Default value if cost_element is not found
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday']])    
            print("***************************************************************************")

Service name:  Cancer Care Services
URL:  https://www.auntbertha.com//desert-care-network---desert-regional-comprehensive-cancer-center--palm-springs-ca--cancer-care-services/5795403837734912?postal=92507
Reviewed on: 11/10/2024
Main Services: ['medical care', 'checkup & test', 'prevent & treat', 'hospital treatment']
Other Services: None
Serving: ['all ages', 'all cancer types', 'breast cancer', 'colon cancer', 'colorectal cancer', 'lung cancer', 'skin cancer', 'benefit recipients', 'low-income']
Phone Number: 760-416-4800
Location Address: 1180 North Indian Canyon Drive, Palm Springs, CA 92262
Location url map: https://www.google.com/maps/?q=1180+North+Indian+Canyon+Drive,+Palm+Springs,+CA+92262/
{'24_hour': True, 'Monday': None, 'Tuesday': None, 'Wednesday': None, 'Thursday': None, 'Friday': None, 'Saturday': None, 'Sunday': None}
Eligibility: []
Latitude: 33.8395185
Longitude: -116.5447073
ZIP Code: 92262
Availability: available
Description: Our Cancer Care Services at Desert Regio

Service name:  Therapy Services
URL:  https://www.auntbertha.com//dr.-bergin-family-counseling-services--riverside-ca--therapy-services/5634769438113792?postal=92507
Reviewed on: 11/09/2024
Main Services: ['counseling', 'specialized therapy', 'group therapy', 'individual counseling']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'all mental health']
Phone Number: 951-684-6684
Location Address: 6800 Indiana Avenue, Riverside, CA 92504
Location url map: https://www.google.com/maps/?q=6800+Indiana+Avenue,+Riverside,+CA+92504/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 33.9428054
Longitude: -117.3915285
ZIP Code: 92504
Availability: available
Description: Therapy Services help people learn how to deal more effectively with situations in the

Service name:  Hospital Services
URL:  https://www.auntbertha.com//menifee-global-medical-center--menifee-ca--hospital-services/6073614944305152?postal=92507
Reviewed on: 11/11/2024
Main Services: ['medical care', 'checkup & test', 'physical therapy', 'occupational therapy', 'speech therapy', 'outpatient treatment']
Other Services: None
Serving: ['all ages', 'all disabilities', 'trauma survivors', 'burn survivors', 'emergency']
Phone Number: 951-679-8888
Location Address: 28400 McCall Boulevard, Menifee, CA 92585
Location url map: https://www.google.com/maps/?q=28400+McCall+Boulevard,+Menifee,+CA+92585/
{'24_hour': True, 'Monday': None, 'Tuesday': None, 'Wednesday': None, 'Thursday': None, 'Friday': None, 'Saturday': None, 'Sunday': None}
Eligibility: []
Latitude: 33.7189951
Longitude: -117.1681517
ZIP Code: 92585
Availability: available
Description: Menifee Global Medical Center provides a full range of specialty services that serve both inpatient and outpatient acute care needs to in

Service name:  Crisis Services
URL:  https://www.auntbertha.com//pathways-to-safety-international--portland-or--crisis-services/6250152036925440?postal=92507
Reviewed on: 11/12/2024
Main Services: ['help escape violence', 'counseling', 'animal welfare', 'navigating the system', 'case management', 'advocacy & legal aid']
Other Services: ['immediate safety', 'counseling', 'animal welfare', 'one-on-one support', 'advocacy & legal aid', 'translation & interpretation']
Serving: ['all ages', 'active duty', 'veterans', 'lgbtqia+', 'trauma survivors', 'abuse or neglect survivors', 'domestic violence survivors', 'human trafficking survivors', 'sexual assault survivors', 'in crisis']
No phone number found.
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM  PST', 'Tuesday': '9:00 AM - 5:00 PM  PST', 'Wednesday': '9:00 AM - 5:00 PM  PST', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
E

Service name:  POMC California Chapter
URL:  https://www.auntbertha.com//parents-of-murdered-children-%2528pomc%2529--san-bernardino-ca--pomc-california-chapter/5699062164094976?postal=92507
Reviewed on: 11/09/2024
Main Services: ['bereavement', 'support network', 'peer support', 'support groups']
Other Services: None
Serving: ['anyone in need', 'all ages', 'families', 'parents']
No phone number found.
Location Address: 2525 North Arrowhead Avenue, San Bernardino, CA 92405
Location url map: https://www.google.com/maps/?q=2525+North+Arrowhead+Avenue,+San+Bernardino,+CA+92405/
{'24_hour': False, 'Monday': '6:30 PM - 8:00 PM  PST', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves parents of children who were victims of murder.']
Latitude: 34.1389889
Longitude: -117.2891501
ZIP Code: 92405
Availability: available
Description: Membership for POMC California Chapter is open to th

...

Service name:  Truck Driver Assistance
URL:  https://www.auntbertha.com//truckersfinalmile--albuquerque-nm--truck-driver-assistance/4591672351653888?postal=92507
Reviewed on: 11/13/2024
Main Services: ['counseling', 'financial assistance']
Other Services: None
Serving: ['all ages', 'grieving', 'individuals', 'families', 'spouses', 'dependents', 'parents', 'in crisis']
Phone Number: 888-857-7871
Location Address: None
Location url map: None
{'24_hour': True, 'Monday': None, 'Tuesday': None, 'Wednesday': None, 'Thursday': None, 'Friday': None, 'Saturday': None, 'Sunday': None}
Eligibility: ['Must be a CDL holder in the course of his/her employment or self employment or an immediate family member of a driver.']
Availability: available
Description: truckersfinalmile provides assistance to North American truck driver's and their families in times of crisis.This program provides:- Help with family reunification- Financial assistance- Grief counselingProgram assistance includes:- Loss of life

Service name:  Problem Gambling Helpline
URL:  https://www.auntbertha.com//the-california-council-on-problem-gambling-%2528ccpg%2529--temecula-ca--problem-gambling-helpline/6283121269407744?postal=92507
Reviewed on: 11/12/2024
Main Services: ['addiction & recovery', 'counseling', 'navigating the system', 'help hotlines', 'support groups']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'seniors', 'families']
Phone Number: 800-426-2537
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Availability: available
Description: The California Council on Problem Gambling operates California’s largest problem gambling helpline, and is free for anyone who is concerned that they, or someone they know, may have a gambling prob

Service name:  Support Groups
URL:  https://www.auntbertha.com//truevolution-inc.--riverside-ca--support-groups/4585696150028288?postal=92507
Reviewed on: 11/14/2024
Main Services: ['counseling', 'peer support', 'support groups', 'virtual support']
Other Services: ['navigating the system']
Serving: ['adults', 'young adults', 'teens', 'seniors', 'lgbtqia+', 'infectious disease', 'hiv/aids', 'limited english']
Phone Number: 877-890-1169
Location Address: 4175 Brockton Avenue, Riverside, CA 92501
Location url map: https://www.google.com/maps/?q=4175+Brockton+Avenue,+Riverside,+CA+92501/
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM  PST', 'Tuesday': '9:00 AM - 5:00 PM  PST', 'Wednesday': '9:00 AM - 5:00 PM  PST', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 4:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves LGBTQ+, and HIV/AIDS positive persons. See the specific details for each group.']
Latitude: 33.9806434
Longitude: -117.3817193


Service name:  Professional Counseling
URL:  https://www.auntbertha.com//crossroads-christian-church--corona-ca--professional-counseling/4821306245775360?postal=92507
Reviewed on: 11/14/2024
Main Services: ['mental health care', 'counseling', 'family counseling', 'individual counseling', 'support network', 'virtual support']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'low-income', 'all mental health']
No phone number found.
Location Address: 2331 Kellogg Avenue, Corona, CA 92881
Location url map: https://www.google.com/maps/?q=2331+Kellogg+Avenue,+Corona,+CA+92881/
{'24_hour': False, 'Monday': '8:00 AM - 4:30 PM', 'Tuesday': '8:00 AM - 4:30 PM', 'Wednesday': '8:00 AM - 4:30 PM', 'Thursday': '8:00 AM - 4:30 PM', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves individuals and families.']
Latitude: 33.8518856
Longitude: -117.5552713
ZIP Code: 92881
Availability: available
Description: Crossroads Christian Church P

Service name:  CSF Surgical Package
URL:  https://www.auntbertha.com//ciruga-sin-fronteras-%2528csf-surgery%2529--bakersfield-ca--csf-surgical-package/6197068244189184?postal=92507
Reviewed on: 11/11/2024
Main Services: ['medical care', 'hospital treatment', 'help pay for healthcare', 'discounted healthcare']
Other Services: ['checkup & test', 'screening & exams']
Serving: ['all ages', 'immigrants', 'undocumented', 'low-income', 'uninsured', 'limited english']
Phone Number: 661-333-5746
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '7:30 AM - 4:30 PM', 'Tuesday': '7:30 AM - 4:30 PM', 'Wednesday': '7:30 AM - 4:30 PM', 'Thursday': '7:30 AM - 4:30 PM', 'Friday': '7:30 AM - 4:30 PM', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['People who do not have medical insurance.', 'Patients who need surgery with a pre-existing medical condition and found to be stable and permissible for surgery.', 'Patients who have been evaluated by a CSF Specialist.', 'Must

In [4]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
csv_filename = "FindHelp_extracted_data_riv_mental_health.csv"

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [5]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
df = pd.read_csv("FindHelp_extracted_data_riv_mental_health.csv")

riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

filtered_df = df[df['Zipcode'].isin(riv_zipcodes)]

filtered_df.to_csv("FindHelp_extracted_data_riv_mental_health.csv", index=False, encoding='utf-8')

filtered_df

,Service_name,Service_url,Main_Services,Other_Services,Serving,Phone_Number,Website,Location_Address,Location_URL_Map,Eligibility,...,Longitude,Zipcode,24hour,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
10,Therapy Services,https://www.auntbertha.com//dr.-bergin-family-...,"['counseling', 'specialized therapy', 'group t...",NaN,"['all ages', 'individuals', 'families', 'all m...",951-684-6684,https://drbergin.net/therapy-services/,"6800 Indiana Avenue, Riverside, CA 92504",https://www.google.com/maps/?q=6800+Indiana+Av...,[],...,-117.391529,92504.0,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
11,Families Counseling,https://www.auntbertha.com//dr.-bergin-family-...,['family counseling'],NaN,"['all ages', 'families', 'in crisis', 'all men...",951-684-6684,https://drbergin.net/therapy-services/,"6800 Indiana Avenue, Riverside, CA 92506",https://www.google.com/maps/?q=6800+Indiana+Av...,[],...,-117.391529,92506.0,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
12,Groups Counseling,https://www.auntbertha.com//dr.-bergin-family-...,"['counseling', 'support network', 'support gro...",NaN,"['adults', 'young adults', 'teens', 'seniors',...",951-684-6684,https://drbergin.net/therapy-services/,"6800 Indiana Avenue, Riverside, CA 92506",https://www.google.com/maps/?q=6800+Indiana+Av...,[],...,-117.391529,92506.0,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
13,Individuals Counseling,https://www.auntbertha.com//dr.-bergin-family-...,['individual counseling'],NaN,"['all ages', 'individuals', 'all mental health']",951-684-6684,https://drbergin.net/therapy-services/,"6800 Indiana Avenue, Riverside, CA 92506",https://www.google.com/maps/?q=6800+Indiana+Av...,[],...,-117.391529,92506.0,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
14,Family Support Services,https://www.auntbertha.com//california-mentor-...,"['mental health care', 'navigating the system']",NaN,"['young adults', 'teens', 'infants', 'children...",951-782-0040,http://www.ca-mentor.com/children-&-families/f...,"6800 Indiana Avenue, Riverside, CA 92506",https://www.google.com/maps/?q=6800+Indiana+Av...,['For Medi-Cal beneficiaries ages 0-21 and the...,...,-117.391529,92506.0,False,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,Closed,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,Residential Substance Abuse Treatement,https://www.auntbertha.com//10-acre-ranch-inc-...,"['addiction & recovery', 'detox', 'residential...","['nutrition education', 'counseling', 'group t...","['substance dependency', 'individuals', 'famil...",866-274-9892,http://www.10acreranch.org,"5953 Grand Avenue, Riverside, CA 92504",https://www.google.com/maps/?q=5953+Grand+Aven...,['This program serves individuals seeking subs...,...,-117.415602,92504.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,Support Groups,https://www.auntbertha.com//truevolution-inc.-...,"['counseling', 'peer support', 'support groups...",['navigating the system'],"['adults', 'young adults', 'teens', 'seniors',...",877-890-1169,https://www.truevolution.org/support-groups,"4175 Brockton Avenue, Riverside, CA 92501",https://www.google.com/maps/?q=4175+Brockton+A...,"['This program serves LGBTQ+, and HIV/AIDS pos...",...,-117.381719,92501.0,False,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 4:00 PM PST,Closed,Closed
551,Integrated Mental Health/Substance Abuse HIV/A...,https://www.auntbertha.com//foothill-aids-proj...,"['outpatient treatment', 'counseling', 'substa...",NaN,"['adults 18+', 'hiv/aids', 'all mental health'...",951-742-7660,https://fapi